In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
# import numpy as np
import pandas as pd

In [2]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os
from degree_days import dds
import autograd.numpy as np




def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [3]:
def get_tensor(df):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    dfc = df.copy()

    df = dfc[energy_cols]

    tensor = df.values.reshape((len(df), 7, stop - start))
    return tensor


def create_region_df_dfc_static(region, year):
    df, dfc = create_matrix_single_region(region, year)
    tensor = get_tensor(df)
    static_region = df[['area', 'total_occupants', 'num_rooms']].copy()
    static_region['area'] = static_region['area'].div(4000)
    static_region['total_occupants'] = static_region['total_occupants'].div(8)
    static_region['num_rooms'] = static_region['num_rooms'].div(8)
    static_region =static_region.values
    return df, dfc, tensor, static_region



In [4]:
au_df, au_dfc, au_tensor, au_static = create_region_df_dfc_static('Austin', year)
sd_df, sd_dfc, sd_tensor, sd_static = create_region_df_dfc_static('SanDiego', year)

In [5]:
import pickle

def load_obj(name):
    with open(os.path.expanduser('~/git/' + name + '.pkl'), 'rb') as f:
        try:
            return pickle.load(f)
        except EOFError:
            return None

In [6]:
load_obj('pred_graph/AtoS/transfer/0-10.0-0')

{'Learning Params': {0: {'Iterations': 900,
   'Lambda': 0,
   'Least Train Error': 11.74557023366184,
   'Num home factors': 4,
   'Num season factors': 2}},
 'Predictions': {'aggregate': [              0           1           2           3           4            5   \
   54    227.370500  228.290730  222.337936  245.054879  351.716943   364.880028   
   203    90.375374   92.579328   78.322375  132.729495  388.185339   419.710952   
   527   599.301642  602.116373  583.908429  653.393246  979.642872  1019.905092   
   1450  434.440339  437.718391  416.513310  497.435747  877.388033   924.277658   
   
                  6            7            8           9           10  \
   54     396.615875   425.529592   424.310910  335.706981  272.214280   
   203    495.718365   564.966772   562.048027  349.841449  197.776312   
   527   1116.976233  1205.415262  1201.687652  930.672844  736.466309   
   1450  1037.327295  1140.323922  1135.982726  820.357243  594.183139   
   
               

In [7]:
pred = {}
for random_seed in range(5):
    for percentage in range(10, 110, 20):
        for parts in range(10):
            p = load_obj('pred_graph/AtoS/transfer/{}-{}-{}'.format(random_seed, float(percentage), parts))

IOError: [Errno 2] No such file or directory: '/Users/xox/git/pred_graph/AtoS/transfer/3-10.0-9.pkl'